# Import Modules
Have to restart kernel after installing DeepStack

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model as kerasload
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys

af_dir = '../../all_faces_bucket/'
disk_data_dir = '../../all_faces_disk/home/jupyter/compressed_forensics_split/'
!pip install git+https://github.com/qubvel/efficientnet

sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/ensembles/DeepStack/deepstack')
from base import *
from ensemble import *
import efficientnet.tfkeras
import joblib
import os
import glob
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# !sudo kill -9 PID # clear GPU memory where 9 is PID number
# !watch -n0.1 nvidia-smi

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-hnvlg21q
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-hnvlg21q
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=31a37ff0a8a1522eb1a2ab2d3b6aca1353dd8f67ef553d222fa6bdfccccacdb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9mfu2rba/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet


Using TensorFlow backend.


# Load models

In [2]:
def get_model(config, network_type):
    '''Loads one of the saved models based on specified config'''
    
    return kerasload(af_dir + 'trained_models/saved_models/ensemble_' + str(config) + '_' + network_type +'.h5')

model1 = get_model(40,'B7')
model2 = get_model(13,'B0')
model3 = get_model(24,'B0')
model4 = get_model(6,'B0')
model5 = get_model(25,'B3')
model6 = get_model(2,'B2')
model7 = get_model(3,'B3')
model8 = get_model(8,'B7')
model9 = get_model(17,'b6')

# Create image generators

In [3]:
import cv2

def perform_compression(x):
    image_type = '.jpg'
    quality_flag = cv2.IMWRITE_JPEG_QUALITY
    quality = 20
    _, encoded_x = cv2.imencode(image_type, x, (int(quality_flag), quality))
    x = cv2.imdecode(encoded_x, cv2.IMREAD_UNCHANGED)
    x = x.astype(float)
    return x


# Create data generators
def augment_data(directory, batch):
    '''Prepares train-time augmentation using given training and validations data)
    
    Returns train_data, val_data'''
    
    datagen1 = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True,
                                 preprocessing_function = perform_compression)
    datagen2 = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)

    # Classes give the folders storing the two different categories
    train_data = datagen1.flow_from_directory('../../forensics_split' + '/validation',
                                             target_size=(224,224), batch_size = batch, classes=['authentic','fake'])
    
    val_data = datagen2.flow_from_directory('../../compressed_forensics_split' + '/test',
                                             target_size=(224,224), batch_size = batch, classes=['authentic','fake'])
    
    return train_data, val_data

train_data, val_data = augment_data('../../forensics_split', batch=128)

Found 21291 images belonging to 2 classes.
Found 23050 images belonging to 2 classes.


# Instantiate KerasMembers
This will find all training and validation predictions. A fit method can then be applied to obtain to the members to create the ensemble.

In [5]:
member1 = KerasMember(name="model1", keras_model=model1, train_batches=train_data, val_batches=val_data)
member2 = KerasMember(name="model2", keras_model=model2, train_batches=train_data, val_batches=val_data)
member3 = KerasMember(name="model3", keras_model=model3, train_batches=train_data, val_batches=val_data)
member4 = KerasMember(name="model4", keras_model=model4, train_batches=train_data, val_batches=val_data)
member5 = KerasMember(name="model5", keras_model=model5, train_batches=train_data, val_batches=val_data)
member6 = KerasMember(name="model6", keras_model=model6, train_batches=train_data, val_batches=val_data)
member7 = KerasMember(name="model7", keras_model=model7, train_batches=train_data, val_batches=val_data)
member8 = KerasMember(name="model8", keras_model=model8, train_batches=train_data, val_batches=val_data)
member9 = KerasMember(name="model9", keras_model=model9, train_batches=train_data, val_batches=val_data)

181/181 [==============================] - 193s 1s/step


In [6]:
# Save Keras members
if not os.path.exists(af_dir + 'trained_models/saved_models/newest_ensemble_members'):
    os.makedirs(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member1.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member2.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member3.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member4.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member5.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member6.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member7.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member8.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')
member9.save(af_dir + 'trained_models/saved_models/newest_ensemble_members')

In [16]:
member1 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model1')
member2 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model2')
member3 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model3')
member4 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model4')
member5 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model5')
member6 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model6')
member7 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model7')
member8 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model8')
member9 = KerasMember.load(af_dir + 'trained_models/saved_models/newest_ensemble_members/model9')

Loaded model1
Loaded model2
Loaded model3
Loaded model4
Loaded model5
Loaded model6
Loaded model7
Loaded model8
Loaded model9


# Fit stacking ensemble

## Compressed training data

In [9]:
stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2, member3, member4, member5, member6, member7, member8, member9])
stack.fit()
stack.describe()

Calling predict
model1 - roc_auc_score: 0.8278
model2 - roc_auc_score: 0.7671
model3 - roc_auc_score: 0.7596
model4 - roc_auc_score: 0.8233
model5 - roc_auc_score: 0.8467
model6 - roc_auc_score: 0.7580
model7 - roc_auc_score: 0.7453
model8 - roc_auc_score: 0.7443
model9 - roc_auc_score: 0.8530
StackEnsemble roc_auc_score: 0.8511


0.8511359790530391

In [10]:
stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2, member3, member4, member5, member6, member7, member8, member9])
stack.fit()
stack.describe(metric='acc')

Calling predict
model1 - Test Accuracy: 0.6412
model2 - Test Accuracy: 0.6200
model3 - Test Accuracy: 0.5849
model4 - Test Accuracy: 0.6812
model5 - Test Accuracy: 0.7193
model6 - Test Accuracy: 0.5860
model7 - Test Accuracy: 0.6439
model8 - Test Accuracy: 0.5796
model9 - Test Accuracy: 0.7601
StackEnsemble Test Accuracy: 0.7724


0.7724078091106291

In [11]:
# Save predictions for video conversion
#np.save(af_dir + 'trained_models/saved_models/ensemble_members/stack_preds', stack.predictions)
np.save(af_dir + 'trained_models/saved_models/ensemble_members/newest_stack_preds', stack.predictions)